### Define Object Classes

In [170]:
class Station():

    def __init__(self,nodeId,name, sttype, mhtime):
        self.nodeId = int(nodeId)
        self.name = name
        self.sttype = sttype
        self.mhtime = mhtime
    
    def __str__(self):
        return f"Station#{self.nodeId} - {self.name} - {self.sttype}"
    
    def getType(self):
        return self.sttype
    
    def getNode(self):
        '''
        returns its node
        '''
        return self.nodeId

In [171]:
class TransportOrder():
    def __init__(self,index,source, dest, ept, ldt, cap,taskType='t'):
        self.id=index
        self.source = source
        self.dest = dest
        self.ept = ept
        self.ldt = ldt
        self.cap=cap
        self.taskType = taskType
        
        
    def __repr__(self):
        return f"TO#{self.id}: From {self.source} to {self.dest}, window: {self.ept} - {self.ldt}, capability: {self.cap}"

In [172]:
class AGV():
    def __init__(self, agvidf, startNode, caps, speed, charge = 40, dischargeRate = 0.5, 
                 chargingRate =1, travelCost =1):
        self.chargingStations=list()
        self.agvidf = agvidf #agv id
        self.startNode = startNode #initial node
        self.caps = caps #capability
        self.speed = speed # speed in m/s
        self.charge = charge # charge %
        self.dischargeRate = dischargeRate # % per second
        self.chargingRate = chargingRate # % per second
        self.taskList = list() # list of tasks
        self.release=(0, startNode) #(time, location)
        self.travelCost = travelCost # weighted travel cost
        self.LOWER_THRESHOLD = 30 # lower threshold of charging
        self.UPPER_THRESHOLD =60 # upper threshold of charging
        self.state = 'N'
        self.lateness = 0 #calculate lateness of jobs
        self.initialSeqFile = ""
        
    def __str__(self):
        return f"AGV#{self.agvidf}, capability:{self.caps}"
    
   

In [173]:
import functools 
def checkCap(to, agvcaps):
        '''
        This function checks if the agv has required capabilities to do the task
        '''
        toCap = to.split(',')
        
        b=[str(c) in agvcaps for c in toCap]
        return functools.reduce(lambda x,y: x and y,b) # reduce does pairwise comparison of 2 objects in a list


## Read Excel Files

In [174]:
from csv import reader

def readDistanceMatrix(distMatrixFile):
    dm = reader(open(distMatrixFile))
    global distMat #used to create global variables from a non-global scope i.e inside a function.
    distMat = list(dm)
    distMat[0][0]=0
    
    
    #print(distMat[1][4])

def getDistanceFromNode(source,dest):
    '''
    returns the distance (in m) between 2 nodes
    '''
    return float(distMat[source][dest])

#read distance matrix
readDistanceMatrix('dm.csv')

## Create main

In [175]:
from csv import reader
from pandas import read_excel


unscheduledTOs = list()
scheduledTOs = dict()
chargingStations = list()
stations = list()


#scheduledTOs["some"]="something"
agvs=list()

def getStationMHT(intId):
    intId = int(intId)
    st = [x for x in stations if x.nodeId == intId]
    
    if st:
        return st[0].mhtime
    

def createAGVs(agvfile):
    df =read_excel(agvfile)
    for index,row in df.iterrows():
        agv = AGV(agvidf=row['agvidf'], startNode=row['startNode'],caps= row['capability'], speed=row['speed'],
                 dischargeRate= row['dischargeRate'], chargingRate = row['chargingRate'],travelCost = row['travelCost'])
        agv.caps+=',Z'
        agvs.append(agv)
        
        print(f'{agv} created')
    

def createRequests(demandfile):
    df =read_excel(demandfile)
    for index,row in df.iterrows():
        transportOrder = TransportOrder(row['Id'], row['source'], row['target'], row['ept'], row['ldt'], row['capability'])
        unscheduledTOs.append(transportOrder)
        print(f'{transportOrder} created')
    for i in range(len(agvs)):
        transportOrder = TransportOrder(999, agvs[len(agvs)-1-i].startNode , agvs[len(agvs)-1-i].startNode, 0, 1000000000, 'Z')
        print('CHARGE INSERTED')
        unscheduledTOs.insert(0,transportOrder)
        
    for i in range(len(chargingStations)):
        for x in range(len(agvs)*5):
            unscheduledTOs.append(TransportOrder(999, chargingStations[i].getNode() , chargingStations[i].getNode(), 0, 1000000000, 'Z'))

def createStations(stationFile):
        df = read_excel(stationFile)
        for index, row in df.iterrows():
                station = Station(row['id'], row['pointidf'], row['type'], row['mhtime'])
                stations.append(station)
                if station.getType() == 'C':
                        chargingStations.append(station)
                print(f"{station} created")

#create stations by reading excel file
createStations(r'stations.xlsx')
#create AGV objects by reading from file
createAGVs(r'agvs.xlsx')
# read demand file and create Transport Orders and add to scheduler list
createRequests(r'trs.xlsx') #use r to avoid errors due to / etc...


   

Station#0 - Dock_1 - PD created
Station#1 - Dock_2 - PD created
Station#2 - Dock_3 - PD created
Station#3 - Dock_4 - PD created
Station#4 - Dock_5 - PD created
Station#5 - Dock_6 - PD created
Station#6 - WH_1 - PD created
Station#7 - WH_2 - PD created
Station#8 - WH_3 - PD created
Station#9 - WH_4 - PD created
Station#10 - WH_5 - PD created
Station#11 - WH_6 - PD created
Station#12 - WH_7 - PD created
Station#13 - WH_8 - PD created
Station#14 - PP_1 - PD created
Station#15 - PP_2 - PD created
Station#16 - PP_3 - PD created
Station#17 - PP_4 - PD created
Station#18 - PP_5 - PD created
Station#19 - PP_6 - PD created
Station#20 - PP_7 - PD created
Station#21 - PP_8 - PD created
Station#22 - PP_9 - PD created
Station#23 - PP_10 - PD created
Station#24 - PP_11 - PD created
Station#25 - PP_12 - PD created
Station#26 - PP_13 - PD created
Station#27 - PP_14 - PD created
Station#28 - PP_15 - PD created
Station#29 - PP_16 - PD created
Station#30 - Tools_1 - PD created
Station#31 - Tools_2 - PD c

In [176]:
unscheduledTOs

[TO#999: From 42 to 42, window: 0 - 1000000000, capability: Z,
 TO#999: From 42 to 42, window: 0 - 1000000000, capability: Z,
 TO#999: From 42 to 42, window: 0 - 1000000000, capability: Z,
 TO#5: From 13 to 4, window: 38 - 1547, capability: C,D,
 TO#9: From 10 to 13, window: 192 - 1673, capability: A,C,
 TO#18: From 22 to 35, window: 281 - 1939, capability: C,D,
 TO#1: From 1 to 22, window: 313 - 1518, capability: E,
 TO#2: From 7 to 21, window: 329 - 963, capability: E,
 TO#3: From 29 to 16, window: 363 - 1919, capability: E,
 TO#15: From 24 to 15, window: 427 - 2242, capability: E,
 TO#19: From 21 to 37, window: 677 - 1956, capability: A,C,
 TO#11: From 15 to 7, window: 694 - 1194, capability: E,
 TO#999: From 42 to 42, window: 0 - 1000000000, capability: Z,
 TO#999: From 42 to 42, window: 0 - 1000000000, capability: Z,
 TO#999: From 42 to 42, window: 0 - 1000000000, capability: Z,
 TO#999: From 42 to 42, window: 0 - 1000000000, capability: Z,
 TO#999: From 42 to 42, window: 0 - 1000

In [177]:
#Gurobi implementation
from gurobipy import *
import gurobipy as grb
import pandas as pd
import numpy as np
import csv

# initialize LP model
# def solveLP(agv):

#Reading distance matrix
filename2=str("dm.csv")
dij = list(csv.reader(open(filename2)))
dij[0][0]=0
for i in range(len(dij)):
    for j in range(len(dij)):
        dij[i][j]=float(dij[i][j])
#Charging Nodes
chargingStns=[x.nodeId for x in chargingStations]

sp=[] #Speed of AGVs
dcr=[] #Discharging rate of AGVs
cr=[] #Charging rate of AGVs
travelCost=[]
startingNode=chargingStns.copy() #Starting node of AGVs (currently set to 18 and 19)
for agv in agvs:
    #Speed of AGV
    sp.append(agv.speed) #mps
    #Battery discharge rate
    dcr.append(agv.dischargeRate)
    #Battery charge rate
    cr.append(agv.chargingRate)
    #travelcosts
    travelCost.append(agv.travelCost)
#----------------------------------------------------------------------------------------------------------------
#GUROBI
m = grb.Model('Scheduling')

#minimization model
m.modelSense = GRB.MINIMIZE

#time limit for optimization
m.setParam("TimeLimit", 600.0);

#decision variables

#variable is 1 if vehicle k goes from request i to j
X={(i,j,k): m.addVar(vtype=GRB.BINARY,name=f"X_{i}_{j}_{k}")
   for i in range(len(unscheduledTOs)) for j in range(len(unscheduledTOs)) for k in range(len(agvs))}

#starting time of job i processing by vehicle k
S={(i,k): m.addVar(vtype=GRB.CONTINUOUS,lb=0.0,name=f"S_{i}_{k}")
   for i in range(len(unscheduledTOs)) for k in range(len(agvs))}

#target reaching time of job i processing by vehicle k
D={(i,k): m.addVar(vtype=GRB.CONTINUOUS,lb=0.0,name=f"D_{i}_{k}")
   for i in range(len(unscheduledTOs)) for k in range(len(agvs))}

#tardiness associated with completion of job i by vehicle j as its kth job
L={(i,k): m.addVar(vtype=GRB.CONTINUOUS,lb=0.0,name=f"L_{i}_{k}")
   for i in range(len(unscheduledTOs)) for k in range(len(agvs))}

#variable to represent the battery status at the beginning of request i of vehicle j
Bs = {(i,k):m.addVar(vtype = GRB.CONTINUOUS, lb = 0.0, ub=100.0, name = f"Bs_{i}_{k}")
     for i in range(len(unscheduledTOs)) for k in range(len(agvs))}

Bf = {(i,k):m.addVar(vtype = GRB.CONTINUOUS, lb = 0.0, ub=100.0, name = f"Bf_{i}_{k}")
     for i in range(len(unscheduledTOs)) for k in range(len(agvs))}

#variable to represent the unloaded distance travelled before the start of a job
U={(i,k): m.addVar(vtype=GRB.CONTINUOUS,lb=0.0,name=f"U_{i}_{k}")
   for i in range(len(unscheduledTOs)) for k in range(len(agvs))}
m.update()
#----------------------------------------------------------------------------------------------------------------
#objective function

#Objective 1 - To minimize lateness
obj_tardiness = grb.quicksum(L[i,k] for i in range(len(unscheduledTOs))  for k in range(len(agvs)))
obj_totalTT = grb.quicksum(U[i,k]*travelCost[k] for i in range(len(agvs),len(unscheduledTOs))  for k in range(len(agvs)))
obj_chargingTask = grb.quicksum(X[i,j,k] if (unscheduledTOs[i].cap=='Z' or unscheduledTOs[j].cap=='Z') else 0 for i in range(len(unscheduledTOs)) for j in range(len(unscheduledTOs)) for k in range(len(agvs)))
alpha = 0.5
m.setObjective(alpha*obj_tardiness+(1-alpha)*obj_totalTT)
#----------------------------------------------------------------------------------------------------------------

#Constraints

#Constraint 1 - each request is processed by only one vehicle
for i in range(len(unscheduledTOs)):
    if not checkCap(unscheduledTOs[i].cap,'Z') or i<len(agvs):
        m.addConstr(grb.quicksum(X[i,j,k] for j in range(len(unscheduledTOs)) for k in range(len(agvs)))==1,name=f"req_{i}")
    else:
        m.addConstr(grb.quicksum(X[i,j,k] for j in range(len(unscheduledTOs)) for k in range(len(agvs)))<=1,name=f"req_{i}")

#Constraint 2 - the first two tasks must be attended by different vehicles
for k in range(len(agvs)):
#     m.addConstr(grb.quicksum(X[i,j,k] for j in range(len(unscheduledTOs)) for i in range(len(agvs)))==1,name=f"firsttwotasks_1_{k}")
    m.addConstr(grb.quicksum(X[j,i,k] for j in range(len(unscheduledTOs)) for i in range(len(agvs)))==0,name=f"firsttwotasks_0_{k}")
    m.addConstr(grb.quicksum(X[k,j,k] for j in range(len(unscheduledTOs)))==1,name=f"AGV{k}_{agvs[k].startNode}")
    m.addConstr(Bs[k,k]>=100-agvs[k].charge, name = f"B0_{k}")
    

#Constraint 3 - Vehicle capability constraints        
for i in range(len(unscheduledTOs)):
    for k in range(len(agvs)):
        if not checkCap(unscheduledTOs[i].cap,agvs[k].caps):
            m.addConstr(grb.quicksum(X[i,j,k] for j in range(len(unscheduledTOs)))==0,name=f"cap_{i}_{k}")
            
#Constraint 4 - Continuity
for i in range(len(unscheduledTOs)):
    if not checkCap(unscheduledTOs[i].cap,'Z'):
        m.addConstr(grb.quicksum(X[j,i,k] for j in range(len(unscheduledTOs)) for k in range(len(agvs)))==1,name=f"reqC_{i}")
    else:
        m.addConstr(grb.quicksum(X[j,i,k] for j in range(len(unscheduledTOs)) for k in range(len(agvs)))<=1,name=f"reqC_{i}")

        
#Constraint - avoiding self - visit
for i in range(len(unscheduledTOs)):
    for k in range(len(agvs)):
        m.addConstr(X[i,i,k]==0,name=f"self_{i}_{k}")

#Constraint continuity
for h in range(len(agvs),len(unscheduledTOs)-len(agvs)):
    for k in range(len(agvs)):
        m.addConstr(grb.quicksum(X[i,h,k] for i in range(len(unscheduledTOs)))-grb.quicksum(X[h,j,k] for j in range(len(unscheduledTOs)))==0,name=f"continuity_{h}_{k}")
        
#Constraint 5 - EPT
for i in range(len(unscheduledTOs)):
    for k in range(len(agvs)):
#         m.addConstr(S[i,k]>=unscheduledTOs[i].ept*grb.quicksum(X[i,j,k] for j in range(len(unscheduledTOs))),name=f"EPT_{i}_{k}")
        m.addConstr(S[i,k]>=unscheduledTOs[i].ept,name=f"EPT_{i}_{k}")
        
#Constraint 6 - LDT
for i in range(len(unscheduledTOs)):
    for k in range(len(agvs)):
        mht = 10 if not checkCap(unscheduledTOs[i].cap,'Z') else 1
        m.addConstr(D[i,k]-L[i,k]<=unscheduledTOs[i].ldt,name=f"LDT_{i}_{k}")
        
#Constraint 7 - Loaded travel time
for i in range(len(unscheduledTOs)):
    for k in range(len(agvs)):
        tsd=(1/sp[k])*dij[unscheduledTOs[i].source][unscheduledTOs[i].dest]
        if not checkCap(unscheduledTOs[i].cap,'Z'):
            m.addConstr(D[i,k]>=S[i,k]+10+tsd,name=f"loaded_travel_{i}_{k}")
        else:
            m.addConstr(D[i,k]>=S[i,k]+tsd+1,name=f"loaded_travel_{i}_{k}")
        
#Constraint 8 - Unloaded travel time
for k in range(len(agvs)):
    for i in range(len(unscheduledTOs)):
        for j in range(len(unscheduledTOs)):
            if i!=j:
                tds=(1/sp[k])*dij[unscheduledTOs[i].dest][unscheduledTOs[j].source]
                m.addConstr(D[i,k]+(10 if not checkCap(unscheduledTOs[i].cap,'Z') else 1)+tds-10000*(1-X[i,j,k])<=S[j,k],name=f"unloaded_{i}_{j}_{k}")     

# Constraint  - Check for conflicts
# for v1 in range(len(agvs)-1):
#     for v2 in range(v1+1,len(agvs)): 
#         for r1 in range(len(unscheduledTOs)-1):
#             for r2 in range(len(unscheduledTOs)-1):
#                 if r1!=r2:
#                     tsdr1=((1/sp[v1])*dij[unscheduledTOs[r1].source][unscheduledTOs[r1].dest])
#                     tsdr2=((1/sp[v2])*dij[unscheduledTOs[r2].source][unscheduledTOs[r2].dest])
#                     if unscheduledTOs[r1].source==unscheduledTOs[r2].source:
#                         if unscheduledTOs[r1].ldt<unscheduledTOs[r2].ldt:
#                             m.addConstr(S[r2,v2]>=D[r1,v1]-tsdr1+1,name=f"ConflictS_v{v1}_{r1}_v{v2}_{r2}")
#                         elif unscheduledTOs[r1].ldt>unscheduledTOs[r2].ldt:
#                             m.addConstr(S[r1,v1]>=D[r2,v2]-tsdr2+1,name=f"Conflict_v{v1}_{r1}_v{v2}_{r2}")
#                     elif unscheduledTOs[r1].dest==unscheduledTOs[r2].dest:
#                         if unscheduledTOs[r1].ldt<unscheduledTOs[r2].ldt:
#                             m.addConstr(D[r2,v2]>=D[r1,v1],name=f"ConflictD_v{v1}_{r1}_v{v2}_{r2}")
#                         elif unscheduledTOs[r1].ldt>unscheduledTOs[r2].ldt:
#                             m.addConstr(D[r1,v1]>=D[r2,v2],name=f"Conflict_v{v1}_{r1}_v{v2}_{r2}")

                            
#Constraint 7 - Check for conflicts
# for v1 in range(len(agvs)-1):
#     for v2 in range(v1+1,len(agvs)): 
#         for r1 in range(len(unscheduledTOs)-1):
#             for r2 in range(len(unscheduledTOs)-1):
#                 if r1!=r2:
#                     if unscheduledTOs[r1].source==unscheduledTOs[r2].source:
#                         tsdr1=((1/sp[v1])*dij[unscheduledTOs[r1].source][unscheduledTOs[r1].dest])
#                         tsdr2=((1/sp[v2])*dij[unscheduledTOs[r2].source][unscheduledTOs[r2].dest])
#                         y=m.addVar(vtype=GRB.BINARY)
#                         m.addConstr(S[r2,v2]+10000*y>=D[r1,v1]-tsdr1+0.1,name=f"ConflictS_v{v1}_{r1}_v{v2}_{r2}")
#                         m.addConstr(S[r1,v1]>=D[r2,v2]-tsdr2+0.1-(1-y)*10000,name=f"Conflict_v{v1}_{r1}_v{v2}_{r2}")
#                     elif unscheduledTOs[r1].dest==unscheduledTOs[r2].dest:
#                         y=m.addVar(vtype=GRB.BINARY)
#                         z=m.addVar(vtype=GRB.CONTINUOUS)
#                         for r3 in range(len(unscheduledTOs)):
#                             m.addGenConstrIndicator(X[r1,r3,v1],True,z==S[r3,v1]-(1/sp[v1])*dij[unscheduledTOs[r1].dest][unscheduledTOs[r3].source])
#                         m.addConstr(D[r2,v2]+10000*y>=z+0.1,name=f"ConflictS_v{v1}_{r1}_v{v2}_{r2}")
#                         w=m.addVar(vtype=GRB.CONTINUOUS)
#                         for r3 in range(len(unscheduledTOs)):
#                             m.addGenConstrIndicator(X[r2,r3,v2],True,w==S[r3,v2]-(1/sp[v2])*dij[unscheduledTOs[r2].dest][unscheduledTOs[r3].source])    
#                         m.addConstr(D[r1,v1]>=w+0.1-(1-y)*10000,name=f"Conflict_v{v1}_{r1}_v{v2}_{r2}")

#total travel calculation
for v in range(len(agvs)):
    for i in range(len(unscheduledTOs)):
        for j in range(len(agvs),len(unscheduledTOs)):
            m.addConstr(U[j,v]>=X[i,j,v]*(1/sp[v])*dij[unscheduledTOs[j].source][unscheduledTOs[i].dest]+\
                        X[i,j,v]*(1/sp[v])*dij[unscheduledTOs[i].source][unscheduledTOs[i].dest],name=f"totaltravel")

                        
#Constraint - battery level                            
for i in range(len(unscheduledTOs)):
    for k in range(len(agvs)):
        if not checkCap(unscheduledTOs[i].cap,'Z'):
            m.addConstr(Bs[i,k]<=70,name=f"batteryub_{i}_{k}")
        else:
            m.addConstr(Bs[i,k]<=100,name=f"batteryub_{i}_{k}")
            
#Battery discharge between nodes
# for a in range(len(unscheduledTOs)):
for i in range(len(unscheduledTOs)):
    for j in range(len(unscheduledTOs)):
        if i!=j:
            for v in range(len(agvs)):
                if not checkCap(unscheduledTOs[i].cap,'Z'):
                    m.addConstr(Bf[i,v]>=Bs[i,v]+dcr[v]*(1/sp[v])*dij[unscheduledTOs[i].source][unscheduledTOs[i].dest]\
                                -cr[v]*(D[i,v]-S[i,v]),name=f"bf_{i}_{j}_{v}")
                else:
                    m.addConstr(Bf[i,v]>=Bs[i,v]+dcr[v]*(1/sp[v])*dij[unscheduledTOs[i].source][unscheduledTOs[i].dest],\
                                name=f"bf_{i}_{j}_{v}")
                m.addConstr(Bs[j,v]>=Bf[i,v]+(dcr[v]*(1/sp[v])*dij[unscheduledTOs[i].dest][unscheduledTOs[j].source])\
                            -10000*(1-X[i,j,v]),f"Battery_{i}_{j}_{v}")
                
            
    
    
#----------------------------------------------------------------------------------------------------------------
#Write model to disk
m.write('model3.lp')
#optimize model
m.update()
m.optimize()
status = m.status
if status == GRB.Status.UNBOUNDED:
    print('The model cannot be solved because it is unbounded')
elif status == GRB.Status.OPTIMAL:
    print('The optimal objective is %g' % m.objVal)
elif status == GRB.Status.INF_OR_UNBD or status == GRB.Status.INFEASIBLE:
    print('Optimization was stopped with status %d' % status)
    # do IIS
    print('The model is infeasible; computing IIS')
    m.computeIIS()
    if m.IISMinimal:
        print('IIS is minimal\n')
    else:
        print('IIS is not minimal\n')
    print('\nThe following constraint(s) cannot be satisfied:')
    for c in m.getConstrs():
        if c.IISConstr:
            print('%s' % c.constrName)
# ------------------------------------------------------------------------------------------------------------------------
print('The optimal objective is %g' % m.objVal)  
for k in range(len(agvs)):
    for i in range(len(unscheduledTOs)):    
        for j in range(len(unscheduledTOs)):            
            if X[i,j,k].x>=0.9:
                print(f"{unscheduledTOs[i].id},{unscheduledTOs[i].source},{unscheduledTOs[i].dest}--{X[i,j,k].varName},{S[i,k].varName},{D[i,k].varName},{L[i,k].varName},{Bs[i,k].varName},{Bf[i,k].varName},{U[i,k].varName} : {X[i,j,k].x:.2f},{S[i,k].x:.2f},{D[i,k].x:.2f},{L[i,k].x:.2f},{100-Bs[i,k].x:.2f},{100-Bf[i,k].x:.2f},{U[i,k].x:.2f}")

print("-----------------------------------KPI------------------------------------------")
print(f"Total tardiness after solving via LP:{sum(D[i,k].x-unscheduledTOs[i].ldt if X[i,j,k].x>0.9 and unscheduledTOs[i].cap!='Z' and D[i,k].x>unscheduledTOs[i].ldt else 0 for i in range(len(unscheduledTOs)) for j in range(len(unscheduledTOs)) for k in range(len(agvs)))} seconds")
print(f"Total unloaded travel distance:{sum(U[j,k].x if X[i,j,k].x>0.9 else 0 for i in range(len(unscheduledTOs)) for j in range(len(unscheduledTOs)) for k in range(len(agvs)))} meters")
print(f"Model run time: {m.runtime}")
      # # solveLP(agvs[0])

Changed value of parameter TimeLimit to 600.0
   Prev: 1e+100  Min: 0.0  Max: 1e+100  Default: 1e+100
Optimize a model with 8809 rows, 2673 columns and 30306 nonzeros
Variable types: 486 continuous, 2187 integer (2187 binary)
Coefficient statistics:
  Matrix range     [2e-02, 1e+04]
  Objective range  [5e-01, 5e-01]
  Bounds range     [1e+00, 1e+02]
  RHS range        [1e+00, 1e+09]
Presolve removed 5261 rows and 1351 columns
Presolve time: 0.05s
Presolved: 3548 rows, 1322 columns, 23261 nonzeros
Variable types: 290 continuous, 1032 integer (1032 binary)

Root relaxation: objective 1.517694e+03, 648 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1517.69373    0   16          - 1517.69373      -     -    0s
     0     0 1579.18627    0   18          - 1579.18627      -     -    0s
     0     0 1580.32845    0   23          - 1580.32845      -    

In [178]:
# totalTT=sum(D[i,k].x-unscheduledTOs[i].ldt if X[i,j,k].x>0.9 and \
#             unscheduledTOs[i].cap!='Z' and D[i,k].x>unscheduledTOs[i].ldt\
#             else 0 for i in range(len(unscheduledTOs)) for j in range(len(unscheduledTOs)) for k in range(len(agvs)))
# TD = sum(U[j,k].x if X[i,j,k].x>0.9 else 0 for i in range(len(unscheduledTOs))\
#          for j in range(len(unscheduledTOs)) for k in range(len(agvs)))

agv0ij=[unscheduledTOs[0].dest,unscheduledTOs[34].source]
agv1ij=[unscheduledTOs[12].dest,unscheduledTOs[33].source]
agv2ij=[unscheduledTOs[13].dest,unscheduledTOs[35].source]
m.objVal-0.5*((dij[agv0ij[0]][agv0ij[1]]/agvs[0].speed)+(dij[agv1ij[0]][agv1ij[1]]/agvs[1].speed)+\
              (dij[agv2ij[0]][agv2ij[1]]/agvs[2].speed))

IndexError: list index out of range

In [ ]:
agvs[1].travelCost

In [ ]:
print(f"Model run time: {m.runtime}")
from matplotlib import pyplot as plt
c=['blue','coral','cyan','black','green','red'] 
#Colors for Loaded Travel Time, Unloaded Travel Time, Loading Time, Unloading Time, Charging Time and Waiting Time

plt.figure(figsize=(20,10))
ls=['solid','dotted']
bY=[] #battery level - y - axis
bX=[] #battery time - x - axis
for v in range(len(agvs)):
    bY.append([])
    bX.append([])
    for r1 in range(len(unscheduledTOs)-2):
        for r2 in range(len(unscheduledTOs)):
            if X[r1,r2,v].x>=0.9:
                plt.plot([S[r1,v].x,S[r1,v].x+10],[unscheduledTOs[r1].source,unscheduledTOs[r1].source],c[2],
                         linestyle=ls[v],label='Loading Time' if 'Loading Time' 
                         not in plt.gca().get_legend_handles_labels()[1] else '')
                plt.plot([S[r1,v].x+10,D[r1,v].x-(1/sp[v])*dij[unscheduledTOs[r1].source][unscheduledTOs[r1].dest]],
                         [unscheduledTOs[r1].source,unscheduledTOs[r1].source],c[5],
                         linestyle=ls[v],label='Waiting Time' if 'Waiting Time'
                         not in plt.gca().get_legend_handles_labels()[1] else '')
                plt.plot([D[r1,v].x-(1/sp[v])*dij[unscheduledTOs[r1].source][unscheduledTOs[r1].dest],D[r1,v].x],
                         [unscheduledTOs[r1].source,unscheduledTOs[r1].dest],c[0],
                         linestyle=ls[v],label='Loaded Travel Time' if 'Loaded Travel Time'
                         not in plt.gca().get_legend_handles_labels()[1] else '')
                plt.plot([D[r1,v].x,D[r1,v].x+(10 if unscheduledTOs[r1].cap!='Z' else 0)],
                         [unscheduledTOs[r1].dest,unscheduledTOs[r1].dest],c[3],
                         linestyle=ls[v],label='Unloading Time' if 'Unloading Time' 
                         not in plt.gca().get_legend_handles_labels()[1] else '')
                if r2<len(unscheduledTOs)-2:
                    plt.plot([D[r1,v].x+(10 if unscheduledTOs[r1].cap!='Z' else 0),S[r2,v].x-(1/sp[v])*dij[unscheduledTOs[r1].dest][unscheduledTOs[r2].source]],
                             [unscheduledTOs[r1].dest,unscheduledTOs[r1].dest],c[5],
                             linestyle=ls[v],label='Waiting Time' if 'Waiting Time' 
                             not in plt.gca().get_legend_handles_labels()[1] else '')
                    plt.plot([S[r2,v].x-(1/sp[v])*dij[unscheduledTOs[r1].dest][unscheduledTOs[r2].source],S[r2,v].x],
                             [unscheduledTOs[r1].dest,unscheduledTOs[r2].source],c[1],
                             linestyle=ls[v],label='Unloaded Travel Time' if 'Unloaded Travel Time' 
                             not in plt.gca().get_legend_handles_labels()[1] else '')
                bY[v].append(100-Bs[r1,v].x)
                bX[v].append(S[r1,v].x)
                if r2<len(unscheduledTOs)-1:
                    bY[v].append(100-Bf[r1,v].x)
                    bX[v].append(D[r1,v].x)


plt.legend(loc='upper left', bbox_to_anchor=(1.05, 0.8))   
plt.xlabel("Time (seconds)")
plt.ylabel("Nodes")
plt.yticks([i for i in range(20)],[i for i in range (20)])  
plt.grid(True)
#Battery status representation - Uncomment the four lines below for battery status
plt.twinx()
plt.ylabel("Battery status")
for v in range(len(agvs)):
    plt.scatter(bX[v],bY[v],label=f"Battery status of AGV {v}",color=c[v])  
plt.yticks([20*i for i in range(1,6)])
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 0.90))
plt.show()

plt.figure(figsize=(20,10))
from matplotlib import pyplot as plt 
for v in range(len(agvs)):
    for r1 in range(len(unscheduledTOs)-2): 
        for r2 in range(len(unscheduledTOs)):
            if unscheduledTOs[r1].id < 999 and X[r1,r2,v].x>0.5:
                i=unscheduledTOs[r1].source
                j=unscheduledTOs[r1].dest
            # Original request window
                plt.plot([unscheduledTOs[r1].ept,unscheduledTOs[r1].ldt-10-dij[i][j]/sp[v]],[i,i],'magenta',
                         linestyle=ls[v], marker=2,label='EPT Window' if 'EPT Window'  
                         not in plt.gca().get_legend_handles_labels()[1] else '')
                plt.plot([unscheduledTOs[r1].ept+10,unscheduledTOs[r1].ldt],[j,j],'cyan',
                         linestyle=ls[v], marker=2,label='LDT Window' if 'LDT Window' 
                         not in plt.gca().get_legend_handles_labels()[1] else '')
                # AGV carries out in

                plt.plot([S[r1,v].x, D[r1,v].x-(1/sp[v])*dij[i][j]],[i+0.5,i+0.5],'red',
                         linestyle=ls[v],marker=2,label='Actual Pick up Time' if 'Actual Pick up Time' 
                         not in plt.gca().get_legend_handles_labels()[1] else '')
                if r2<len(unscheduledTOs)-2:
                    plt.plot([D[r1,v].x, S[r2,v].x-(1/sp[v])*dij[j][unscheduledTOs[r2].source]],[j+0.5,j+0.5],'blue',linestyle=ls[v],
                              marker=2,label='Actual Delivery Time' if 'Actual Delivery Time' 
                              not in plt.gca().get_legend_handles_labels()[1] else '')
plt.legend(loc='upper left', bbox_to_anchor=(1.05, 0.8))   
plt.yticks([i for i in range(20)],[i for i in range (20)])
plt.ylabel("Node")
plt.xlabel("Time(Seconds)")
plt.grid(True)
plt.show()



In [ ]:
# for k in range(len(agvs)):
#     for i in range(len(unscheduledTOs)):    
#         for j in range(len(unscheduledTOs)):            
#             if X[i,j,k].x>=0:
#                 print(f"{unscheduledTOs[i].id},{unscheduledTOs[i].source},{unscheduledTOs[i].dest}--{X[i,j,k].varName},{S[i,k].varName},{D[i,k].varName},{L[i,k].varName},{Bs[i,k].varName},{Bf[i,k].varName} : {X[i,j,k].x},{S[i,k].x},{D[i,k].x},{L[i,k].x},{100-Bs[i,k].x},{100-Bf[i,k].x}")
print(f"{0.243:.2f}")

In [ ]:
agvs[0].dischargeRate